In [1]:
!ls data/han/AWS/

ls: 'data/han/AWS/'에 접근할 수 없습니다: 그런 파일이나 디렉터리가 없습니다


In [2]:
ls data_test/한강1/자동

가평_2016.xlsx  서상_2016.xlsx  의암호_2016.xlsx  화천_2016.xlsx
가평_2017.xlsx  서상_2017.xlsx  의암호_2017.xlsx  화천_2017.xlsx
가평_2018.xlsx  서상_2018.xlsx  의암호_2018.xlsx  화천_2018.xlsx
가평_2019.xlsx  서상_2019.xlsx  의암호_2019.xlsx  화천_2019.xlsx


In [3]:
!pwd

/home/jaewon/work/water-quality/rnn


In [4]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [5]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [6]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [33]:

# folder = 'data/han/자동'
# folder = 'data/han/수질'
#folder = 'data/han/AWS'
# folder = 'data/nakdong/자동'

## 한강
## 자동

#folder = 'data_test/한강1/자동/'
folder = 'data_test/한강1/AWS_T/'
#folder = 'data/'

#file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#              ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
              ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
              ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]


    
#file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#             ]

    



## 낙동강
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]
             



In [34]:
target_col = '총유기탄소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:11', ':,28:45', ':,26:29']
nakdong = [':,2:11', ':,2:12']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = han[2]
#iloc_val = han[0]
iloc_val

':,26:29'

In [35]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [36]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [37]:
def normalize(df):
    # normalize data
    
    
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    return df_all, train_mean, train_std

In [39]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)

남이섬_2016.xlsx
(8784, 30)
(8784, 30)
남이섬_2017.xlsx
(8760, 30)
(8760, 30)
남이섬_2018.xlsx
(8760, 30)
(8760, 30)
남이섬_2018.xlsx
(8760, 30)
(8760, 30)
청평_2016.xlsx
(8784, 30)
(8784, 30)
청평_2017.xlsx
(8760, 30)
(8760, 30)
청평_2018.xlsx
(8760, 30)
(8760, 30)
청평_2019.xlsx
(8760, 30)
(8760, 30)
화천_2016.xlsx
(8784, 30)
(8784, 30)
화천_2017.xlsx
(8760, 30)
(8760, 30)
화천_2018.xlsx
(8760, 30)
(8760, 30)
화천_2019.xlsx
(8760, 30)
(8760, 30)


In [40]:
train_mean, train_std

(rn60m_value    1.389388e-01
 ta_value       1.109134e+01
 hm_value       7.313731e+01
 Day sin        4.971771e-15
 Day cos       -1.358341e-15
 Year sin      -2.876199e-07
 Year cos       2.053419e-05
 dtype: float64,
 rn60m_value     1.123228
 ta_value       11.634825
 hm_value       23.012861
 Day sin         0.707110
 Day cos         0.707110
 Year sin        0.707103
 Year cos        0.707117
 dtype: float64)

In [41]:
df[2]

,rn60m_value,ta_value,hm_value,Day sin,Day cos,Year sin,Year cos
0,NaN,NaN,NaN,-1.998079e-12,1.414207,-0.003771,1.414158
1,NaN,NaN,NaN,3.660237e-01,1.366019,-0.002757,1.414161
2,NaN,NaN,NaN,7.071034e-01,1.224739,-0.001743,1.414162
3,NaN,NaN,NaN,9.999952e-01,0.999995,-0.000729,1.414163
4,NaN,NaN,NaN,1.224739e+00,0.707103,0.000284,1.414163
...,...,...,...,...,...,...,...
35059,-0.123696,-1.683853,-0.788138,-1.366019e+00,0.366024,-0.008109,1.414140
35060,-0.123696,-1.744017,-0.814210,-1.224739e+00,0.707103,-0.007095,1.414145
35061,-0.123696,-1.804182,-0.475269,-9.999952e-01,0.999995,-0.006082,1.414150
35062,-0.123696,-1.829966,-0.362289,-7.071034e-01,1.224739,-0.005068,1.414154


In [42]:
df[2]

,rn60m_value,ta_value,hm_value,Day sin,Day cos,Year sin,Year cos
0,NaN,NaN,NaN,-1.998079e-12,1.414207,-0.003771,1.414158
1,NaN,NaN,NaN,3.660237e-01,1.366019,-0.002757,1.414161
2,NaN,NaN,NaN,7.071034e-01,1.224739,-0.001743,1.414162
3,NaN,NaN,NaN,9.999952e-01,0.999995,-0.000729,1.414163
4,NaN,NaN,NaN,1.224739e+00,0.707103,0.000284,1.414163
...,...,...,...,...,...,...,...
35059,-0.123696,-1.683853,-0.788138,-1.366019e+00,0.366024,-0.008109,1.414140
35060,-0.123696,-1.744017,-0.814210,-1.224739e+00,0.707103,-0.007095,1.414145
35061,-0.123696,-1.804182,-0.475269,-9.999952e-01,0.999995,-0.006082,1.414150
35062,-0.123696,-1.829966,-0.362289,-7.071034e-01,1.224739,-0.005068,1.414154


In [43]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [44]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [45]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [46]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [47]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [48]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [49]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [50]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [51]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [52]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [53]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [54]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [55]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [56]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[   0   13   13    0]
 [  13   11    4   13]
 [  17   18   18   24]
 [  35    2    1   42]
 [  36    9    9   44]
 [  45   23   23   53]
 [  68   21   21   76]
 [  89   16   16   97]
 [ 105   19   19  113]
 [ 124   14   14  132]
 [ 138    2    1  146]
 [ 139   15    6  148]
 [ 145   10   10  163]
 [ 155   14   14  173]
 [ 169   15   15  187]
 [ 184   23    8  202]
 [ 192    2    1  225]
 [ 193   19   19  227]
 [ 212    4    4  246]
 [ 216   16   16  250]
 [ 232   12   12  266]
 [ 244   11    4  278]
 [ 248   12   12  289]
 [ 260   53   19  301]
 [ 279   14   14  354]
 [ 293   14   14  368]
 [ 307   15   15  382]
 [ 322   23    8  397]
 [ 330    2    1  420]
 [ 331   19   19  422]
 [ 350    4    4  441]
 [ 354   16   16  445]
 [ 370   12   12  461]
 [ 382   11    4  473]
 [ 386   12   12  484]
 [ 398   53   19  496]
 [ 417   14   14  549]
 [ 431    3    3  563]
 [ 434   16   16  566]
 [ 450   10   10  582]
 [ 460    6    6  592]
 [ 466   20   20  598]
 [ 486   13   13  618]
 [ 499   17

In [57]:
dgen = GainDataGenerator(df)

pattern none


In [58]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,rn60m_value,ta_value,hm_value,Day sin,Day cos,Year sin,Year cos
0,NaN,NaN,NaN,-1.407890e-12,1.000000,-0.002666,0.999996
1,NaN,NaN,NaN,2.588190e-01,0.965926,-0.001950,0.999998
2,NaN,NaN,NaN,5.000000e-01,0.866025,-0.001233,0.999999
3,NaN,NaN,NaN,7.071068e-01,0.707107,-0.000516,1.000000
4,NaN,NaN,NaN,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...
35059,0.0,-8.5,55.0,-9.659258e-01,0.258819,-0.005734,0.999984
35060,0.0,-9.2,54.4,-8.660254e-01,0.500000,-0.005017,0.999987
35061,0.0,-9.9,62.2,-7.071068e-01,0.707107,-0.004301,0.999991
35062,0.0,-10.2,64.8,-5.000000e-01,0.866025,-0.003584,0.999994


In [59]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

In [ ]:
## Graph 

In [60]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 7])

In [61]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: '총유기탄소'

In [62]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [63]:
val_performance = {}
performance = {}

In [64]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [65]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [66]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 696ms/step - gen_loss: 124.1747 - disc_loss: 0.7285 - rmse: 1.0059 - val_loss: 0.9462
Epoch 2/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 90.7668 - disc_loss: 0.5704 - rmse: 0.9327 - val_loss: 0.9543
Epoch 3/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 79.3296 - disc_loss: 0.4265 - rmse: 1.0566 - val_loss: 0.8637
Epoch 4/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 76.5343 - disc_loss: 0.3190 - rmse: 0.8263 - val_loss: 0.8850
Epoch 5/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 58.6792 - disc_loss: 0.3377 - rmse: 0.8870 - val_loss: 1.1521
Epoch 6/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 50.3306 - disc_loss: 0.3442 - rmse: 1.0082 - val_loss: 0.8941
Epoch 7/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 40.1535 - disc_loss: 0.3146 - rmse: 0.8941 - val_loss: 0.8770
Epoch

1/1 [==============================] - 0s 80ms/step - gen_loss: 10.3266 - disc_loss: 0.2380 - rmse: 0.7576 - val_loss: 1.0398
Epoch 60/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 12.4171 - disc_loss: 0.2355 - rmse: 0.9922 - val_loss: 0.6450
Epoch 61/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 11.4303 - disc_loss: 0.2328 - rmse: 0.5817 - val_loss: 0.7520
Epoch 62/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 10.1776 - disc_loss: 0.2333 - rmse: 0.4869 - val_loss: 0.7417
Epoch 63/2000
1/1 [==============================] - 0s 79ms/step - gen_loss: 8.7463 - disc_loss: 0.2310 - rmse: 0.8198 - val_loss: 0.8294
Epoch 64/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 7.8467 - disc_loss: 0.2308 - rmse: 0.4232 - val_loss: 0.7198
Epoch 65/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 9.0057 - disc_loss: 0.2277 - rmse: 0.8926 - val_loss: 0.7229
Epoch 66/2000
1/1 [==================

1/1 [==============================] - 0s 82ms/step - gen_loss: 9.7051 - disc_loss: 0.1545 - rmse: 0.6448 - val_loss: 0.7085
Epoch 118/2000
1/1 [==============================] - 0s 86ms/step - gen_loss: 6.8777 - disc_loss: 0.1597 - rmse: 0.7372 - val_loss: 0.6763
Epoch 119/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 6.0564 - disc_loss: 0.1564 - rmse: 0.5770 - val_loss: 0.9969
Epoch 120/2000
1/1 [==============================] - 0s 86ms/step - gen_loss: 14.6116 - disc_loss: 0.1544 - rmse: 0.6490 - val_loss: 0.6229
Epoch 121/2000
1/1 [==============================] - 0s 85ms/step - gen_loss: 7.2593 - disc_loss: 0.1599 - rmse: 0.6602 - val_loss: 0.8936
Epoch 122/2000
1/1 [==============================] - 0s 85ms/step - gen_loss: 5.5630 - disc_loss: 0.1489 - rmse: 0.6421 - val_loss: 0.6612
Epoch 123/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 6.1082 - disc_loss: 0.1547 - rmse: 0.9371 - val_loss: 0.8801
Epoch 124/2000
1/1 [==============

1/1 [==============================] - 0s 81ms/step - gen_loss: 7.8227 - disc_loss: 0.1295 - rmse: 0.6226 - val_loss: 0.8117
Epoch 176/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 7.7577 - disc_loss: 0.1243 - rmse: 0.8271 - val_loss: 0.8208
Epoch 177/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 6.3369 - disc_loss: 0.1264 - rmse: 0.7691 - val_loss: 0.8077
Epoch 178/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 6.9245 - disc_loss: 0.1300 - rmse: 0.9129 - val_loss: 0.6085
Epoch 179/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 7.5157 - disc_loss: 0.1275 - rmse: 0.7592 - val_loss: 0.7039
Epoch 180/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 5.7134 - disc_loss: 0.1287 - rmse: 0.9076 - val_loss: 1.0214
Epoch 181/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 9.9604 - disc_loss: 0.1247 - rmse: 0.8469 - val_loss: 0.7363
Epoch 182/2000
1/1 [===============

1/1 [==============================] - 0s 79ms/step - gen_loss: 5.2026 - disc_loss: 0.1204 - rmse: 0.6167 - val_loss: 0.6442
Epoch 234/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 4.7119 - disc_loss: 0.1169 - rmse: 0.6375 - val_loss: 0.7038
Epoch 235/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 4.6938 - disc_loss: 0.1171 - rmse: 0.7078 - val_loss: 1.0090
Epoch 236/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 5.9291 - disc_loss: 0.1152 - rmse: 0.8168 - val_loss: 0.9504
Epoch 237/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 6.8529 - disc_loss: 0.1181 - rmse: 0.6353 - val_loss: 0.7535
Epoch 238/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 4.9469 - disc_loss: 0.1198 - rmse: 0.6341 - val_loss: 0.8529
Epoch 239/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 8.1859 - disc_loss: 0.1176 - rmse: 0.6567 - val_loss: 0.7713
Epoch 240/2000
1/1 [===============

1/1 [==============================] - 0s 81ms/step - gen_loss: 6.8600 - disc_loss: 0.1088 - rmse: 0.7818 - val_loss: 0.7494
Epoch 292/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 7.5886 - disc_loss: 0.1102 - rmse: 1.2319 - val_loss: 0.7091
Epoch 293/2000
1/1 [==============================] - 0s 79ms/step - gen_loss: 5.9009 - disc_loss: 0.1140 - rmse: 1.0779 - val_loss: 0.7215
Epoch 294/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 5.2635 - disc_loss: 0.1147 - rmse: 0.7844 - val_loss: 0.7487
Epoch 295/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 5.7995 - disc_loss: 0.1110 - rmse: 0.9071 - val_loss: 0.8467
Epoch 296/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 4.8165 - disc_loss: 0.1124 - rmse: 0.7651 - val_loss: 1.0193
Epoch 297/2000
1/1 [==============================] - 0s 79ms/step - gen_loss: 8.8042 - disc_loss: 0.1083 - rmse: 0.9261 - val_loss: 0.7332
Epoch 298/2000
1/1 [===============

1/1 [==============================] - 0s 80ms/step - gen_loss: 6.9280 - disc_loss: 0.1095 - rmse: 1.1442 - val_loss: 0.7002
Epoch 350/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 4.5071 - disc_loss: 0.1090 - rmse: 1.0167 - val_loss: 0.8045
Epoch 351/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 4.4252 - disc_loss: 0.1109 - rmse: 0.6379 - val_loss: 0.9025
Epoch 352/2000
1/1 [==============================] - 0s 79ms/step - gen_loss: 6.4916 - disc_loss: 0.1050 - rmse: 0.9541 - val_loss: 0.5898
Epoch 353/2000
1/1 [==============================] - 0s 83ms/step - gen_loss: 6.6381 - disc_loss: 0.1109 - rmse: 0.9414 - val_loss: 0.7241
Epoch 354/2000
1/1 [==============================] - 0s 84ms/step - gen_loss: 4.0246 - disc_loss: 0.1044 - rmse: 0.8453 - val_loss: 0.7352
Epoch 355/2000
1/1 [==============================] - 0s 84ms/step - gen_loss: 6.7716 - disc_loss: 0.1095 - rmse: 0.6625 - val_loss: 0.8283
Epoch 356/2000
1/1 [===============

1/1 [==============================] - 0s 82ms/step - gen_loss: 4.5741 - disc_loss: 0.1049 - rmse: 0.6584 - val_loss: 0.9370
Epoch 408/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 3.6944 - disc_loss: 0.1047 - rmse: 0.8610 - val_loss: 1.0122
Epoch 409/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 3.6465 - disc_loss: 0.1023 - rmse: 0.6506 - val_loss: 0.5368
Epoch 410/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 8.7279 - disc_loss: 0.1059 - rmse: 0.9829 - val_loss: 0.7910
Epoch 411/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 4.7394 - disc_loss: 0.1068 - rmse: 0.7339 - val_loss: 0.6572
Epoch 412/2000
1/1 [==============================] - 0s 83ms/step - gen_loss: 5.2410 - disc_loss: 0.1043 - rmse: 0.8611 - val_loss: 0.4767
Epoch 413/2000
1/1 [==============================] - 0s 87ms/step - gen_loss: 4.1055 - disc_loss: 0.1041 - rmse: 0.9249 - val_loss: 0.9320
Epoch 414/2000
1/1 [===============

1/1 [==============================] - 0s 81ms/step - gen_loss: 4.3769 - disc_loss: 0.1028 - rmse: 0.8821 - val_loss: 0.6007
Epoch 466/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 3.0390 - disc_loss: 0.0985 - rmse: 0.6694 - val_loss: 0.6675
Epoch 467/2000
1/1 [==============================] - 0s 82ms/step - gen_loss: 5.0738 - disc_loss: 0.1031 - rmse: 0.8370 - val_loss: 0.7643
Epoch 468/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 5.9804 - disc_loss: 0.1019 - rmse: 0.8164 - val_loss: 1.0265
Epoch 469/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 3.6548 - disc_loss: 0.1048 - rmse: 1.0019 - val_loss: 0.7005
Epoch 470/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 3.5465 - disc_loss: 0.1035 - rmse: 0.6239 - val_loss: 0.9827
Epoch 471/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 4.0391 - disc_loss: 0.1028 - rmse: 0.8655 - val_loss: 0.7049
Epoch 472/2000
1/1 [===============

1/1 [==============================] - 0s 80ms/step - gen_loss: 3.3081 - disc_loss: 0.1036 - rmse: 0.6610 - val_loss: 0.6442
Epoch 524/2000
1/1 [==============================] - 0s 79ms/step - gen_loss: 4.0781 - disc_loss: 0.1042 - rmse: 0.6248 - val_loss: 0.9516
Epoch 525/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 3.5371 - disc_loss: 0.0986 - rmse: 0.6661 - val_loss: 0.8816
Epoch 526/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 4.9213 - disc_loss: 0.1011 - rmse: 0.7640 - val_loss: 0.6399
Epoch 527/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 3.5412 - disc_loss: 0.1009 - rmse: 0.6710 - val_loss: 0.7588
Epoch 528/2000
1/1 [==============================] - 0s 80ms/step - gen_loss: 5.4744 - disc_loss: 0.1028 - rmse: 0.7414 - val_loss: 0.8864
Epoch 529/2000
1/1 [==============================] - 0s 81ms/step - gen_loss: 5.6313 - disc_loss: 0.1017 - rmse: 0.7718 - val_loss: 0.6889
Epoch 530/2000
1/1 [===============

**학습 loss history 출력**

In [67]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [68]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 1s 15ms/step - loss: 0.8732


0.8732386231422424

**model save**

In [69]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [70]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: '총유기탄소'

In [71]:
norm_df = pd.concat(df,axis=0)

In [72]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 7)
120
x.shape = (105120, 7)
x_reshape.shape = (876, 120, 7)
81507
y_true.shape= (105120, 7)
81507


(1, 120, 7)

In [73]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 3ms/step - loss: 0.0000e+00


0.0

In [74]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [75]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 7) (120, 7)
(105192, 7)


In [76]:
y_pred[~np.isnan(data)] = np.nan

In [77]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: index 7 is out of bounds for axis 1 with size 7

**Imputed Data**

In [78]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 7)
120
n= 105120
(35040, 3)
(35040, 3)


# 예측

In [79]:
df[0].head

<bound method NDFrame.head of        rn60m_value  ta_value  hm_value       Day sin   Day cos  Year sin  \
0              NaN       NaN       NaN -1.998079e-12  1.414207 -0.003771   
1              NaN       NaN       NaN  3.660237e-01  1.366019 -0.002757   
2              NaN       NaN       NaN  7.071034e-01  1.224739 -0.001743   
3              NaN       NaN       NaN  9.999952e-01  0.999995 -0.000729   
4              NaN       NaN       NaN  1.224739e+00  0.707103  0.000284   
...            ...       ...       ...           ...       ...       ...   
35059    -0.123696 -1.262704 -1.083625 -1.366019e+00  0.366024 -0.002209   
35060    -0.123696 -1.383032 -0.714266 -1.224739e+00  0.707103 -0.001196   
35061    -0.123696 -1.417412 -0.788138 -9.999952e-01  0.999995 -0.000182   
35062    -0.123696 -1.554930 -0.531760 -7.071034e-01  1.224739  0.000832   
35063    -0.123696 -1.658069 -0.214546 -3.660237e-01  1.366019  0.001845   

       Year cos  
0      1.414158  
1      1.414161  
2  

In [80]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

(35040, 7) (35040, 7)
(35040, 3) (35040, 3)
(35040, 13) (35040, 13)


In [81]:
df[0].columns

Index(['rn60m_value', 'ta_value', 'hm_value', 'Day sin', 'Day cos', 'Year sin',
       'Year cos'],
      dtype='object')

In [82]:
#### target_col = '총질소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
out_num_features = len(out_features)
out_num_features

KeyError: '총유기탄소'

**예측결과 class**

In [83]:
cond = {
    "총인":[
        [-1000,0.02],
        [0.02,0.04],
        [0.04,0.10],
        [0.10,0.20],
        [0.20,0.30],
        [0.30,0.50],
        [0.50,1000],
    ],
    "총질소":[
        [-1000,0.2],
        [0.2,0.3],
        [0.3,0.4],
        [0.4,0.6],
        [0.6,1.0],
        [1.0,1.5],
        [1.5,1000],
    ],
    "수소이온농도":[
        [6.5 ,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.0,8.5],
        [6.0,8.5],
        [-1000,1000],
    ],
    "총유기탄소":[
        [-1000 ,2],
        [2,3],
        [3,4],
        [4,5],
        [5,6],
        [6,8],
        [8,1000],
    ],
    "용존산소":[
        [7.5 ,1000],
        [5,7.5],
        [5,7.5],
        [5,7.5],
        [2,5],
        [2,5],
        [-1000,2],
    ],
    "클로로필-a":[
        [-1000 ,5],
        [5,9],
        [9,14],
        [14,20],
        [20,42],
        [42,70],
        [70,1000],
    ],
    
}

In [84]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [85]:
OUT_STEPS = 24*3
wdg = WaterDataGenerator(train_df,
                         batch_size=128,
                         input_width = 24*7,
                         label_width = OUT_STEPS,
                         shift = OUT_STEPS,
                         out_num_features = out_num_features,
#                          out_features = out_features
                        )

NameError: name 'out_num_features' is not defined

In [86]:
train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.051727,-1.041916,-0.165526,-1.998079e-12,1.414207,-0.003771,1.414158,-0.048327,-1.042207,-0.165907,-0.045945,-1.040336,-0.163955
1,-0.365352,-0.984972,0.050321,3.660237e-01,1.366019,-0.002757,1.414161,-0.366114,-0.983925,0.050618,-0.362314,-0.982401,0.052820
2,0.152969,-1.115919,0.057290,7.071034e-01,1.224739,-0.001743,1.414162,0.150149,-1.114321,0.054748,0.152779,-1.113582,0.058771
3,-0.051526,-0.948034,0.108115,9.999952e-01,0.999995,-0.000729,1.414163,-0.054476,-0.946739,0.106351,-0.053547,-0.946506,0.111485
4,-0.055546,-0.819190,-0.100641,1.224739e+00,0.707103,0.000284,1.414163,-0.055045,-0.818298,-0.101703,-0.052699,-0.817179,-0.096230
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,-0.123696,0.224212,-0.401398,-1.366019e+00,0.366024,-1.352624,0.412795,-0.123696,0.172642,-0.479615,-0.123696,-0.128179,0.524172
24524,-0.123696,0.000744,0.224339,-1.224739e+00,0.707103,-1.352327,0.413764,-0.123696,0.086693,-0.410089,-0.123696,-0.188343,0.711024
24525,-0.123696,-0.179748,0.750132,-9.999952e-01,0.999995,-1.352030,0.414733,-0.123696,-0.016445,-0.062457,-0.123696,-0.300077,0.867458
24526,-0.123696,-0.214128,0.984784,-7.071034e-01,1.224739,-1.351733,0.415702,-0.123696,-0.110989,0.350356,-0.123696,-0.334456,0.954366


In [87]:
it = iter(wdg)

NameError: name 'wdg' is not defined

In [88]:
x,y = next(it)
x.shape, y.shape

NameError: name 'it' is not defined

In [89]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [90]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [91]:
def hour_to_day_mean(array):
    time = 24
    return tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [92]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plot_out_col_index = self.column_indices[plot_out_col]

  if self.label_columns:
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_out_col_index = plot_out_col_index
    
    
  if model is None:
    return
    
  print("column : " + df_all.columns[plot_col])
    
  count1 = 0
  count2 = 0
  for n in range(len(inputs)):
    predictions = model(inputs)
    pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
    pred_temp = pred_temp * train_std[plot_col] + train_mean[plot_col]
    label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    level_label = 0 
    level_pred = 0
    
    for i in range(0,7,1):
      min, max = cond[df_all.columns[plot_col]][i]
      if min <= label_temp[2] and label_temp[2] < max:
        level_label = i
            #print(i, label_temp[2])
      if min <= pred_temp[2] and pred_temp[2] < max:
        level_pred = i
            
    if level_pred != level_label:
        count1 = count1 + 1
    count2 = count2 + 1 
        
  print(100 - count1*100/count2)

  return 100 - count1*100/count2
    

WindowGenerator.compa = compa

In [93]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
multi_window

Total window size: 240
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167]
Label indices: [168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 

In [94]:
out_features

NameError: name 'out_features' is not defined

In [95]:
MAX_EPOCHS = 400

def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [96]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

**Baseline model**

In [97]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['BaseLine'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['BaseLine'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['BaseLine'])
print('test performance = ', multi_performance['BaseLine'])


NameError: name 'out_features' is not defined

In [98]:
multi_window.plot2(last_baseline, plot_col=out_features[0])

NameError: name 'out_features' is not defined

**Linear**

In [99]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

NameError: name 'out_num_features' is not defined

In [ ]:
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_linear_model, plot_col=out_features[0])

**Elman_RNN**

In [ ]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(elman_model, plot_col=out_features[0])

In [ ]:
# target_col = '수온'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
out_num_features = len(out_features)
out_num_features

In [ ]:
out_features[0]

In [ ]:
multi_window.plot2(elman_model, plot_col=14)

**GRU**

In [ ]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
MAX_EPOCHS = 450

In [ ]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(gru_model, plot_col=3)

In [ ]:
multi_window.plot2(gru_model, plot_col=out_features[0])

**LSTM**

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_lstm_model, plot_col=out_features[0])

**CNN model**

In [ ]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [ ]:
MAX_EPOCHS = 400
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

In [ ]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_conv_model, plot_col=out_features[0])

**performance**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()